In [95]:
import pandas as pd
import requests
import numpy as np
import altair as alt
from countrycode import countrycode

## Turning MIP Dataset into GeoJSON

In [97]:
# Military intervention data

df = pd.read_csv("MIP-Dataset-2022-JK.csv")

In [98]:
# Pulling relevant rows and columns from database. We're only looking at data after WWII

df = df[(df['styear'] > 1944) & (df['RemoveCase']==0)][['State B', 'State B code', 'styear', 'endyear', 'Name', 'Description', 'Era', 'PresName', 'Objective', 
                              'ForeignReg', 'BuildReg', 'Empire', 'Territory', 'Policy', 'SocialProt', 'Economic', 'Fatalities', 'UNRegN', 'UNRegT', 'PresElectionYear']]

In [99]:
# Converting Correlates of War (CoW) codes to ISO alpha-2 two-letter country codes

df['State B code'].dropna().astype(int)
df['iso2c'] = df['State B code'].apply(lambda x: countrycode([x], origin='cown', destination='iso2c')[0])

In [100]:
# Adding in country names based on ISO alpha-2 codes

def get_name(country): 
    response = requests.get(f"http://api.geonames.org/countryInfoJSON?country={country}&username=jwokime")
    info = response.json()
    try:
        name = info['geonames'][0]['countryName']
    except (IndexError, KeyError): 
        name = None
    return name

df['countryName'] = df['iso2c'].apply(lambda x: get_name(x))

In [101]:
# Adding in coordinates based on country names
key = 'b9c6daada0e84575b0f8e8e859caf853'

def get_lat(country): 
    url = f'https://api.opencagedata.com/geocode/v1/json?q=countrycode%3D{country}&key={key}&language=en&pretty=1'
    info = requests.get(url).json()
    try:
        lat = info['results'][0]['geometry']['lat']
    except (IndexError, KeyError):
        lat = None
    return lat

def get_long(country): 
    url = f'https://api.opencagedata.com/geocode/v1/json?q=countrycode%3D{country}&key={key}&language=en&pretty=1'
    info = requests.get(url).json()
    try:
        long = float(info['results'][0]['geometry']['lng'])
    except (IndexError, KeyError):
        long = None
    return long

df['lat'] = df['countryName'].apply(lambda x: get_lat(x))
df['long'] = df['countryName'].apply(lambda x: get_long(x))

In [102]:
# GeoNames API doesn't have information for defunct nations, so manually adding in coordinates and names
df['iso2c'] = df['iso2c'].fillna("YU")
df['countryName'] = df['countryName'].fillna("Yugoslavia")                                 
df.loc[df['iso2c'] == 'YU', 'lat'] = 43.85
df.loc[df['iso2c'] == 'YU', 'long'] = 18.2

In [25]:
# Add offsets to the duplicate coordinates so that they are visible when put on a map
def add_offset(lat, long, magnitude=0.001):
    offset = np.random.uniform(-magnitude, magnitude)
    return lat + offset, long + offset

duplicates = df.duplicated(subset=['lat', 'long'], keep=False)

for index, row in df[duplicates].iterrows():
    lat, lon = add_offset(row['lat'], row['long'])
    df.at[index, 'lat'] = lat
    df.at[index, 'long'] = lon

In [28]:
# Convert to GeoJSON
import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat))

interventions_geojson = gdf.to_json()

with open('data/interventions.geojson', 'w') as f:
    f.write(interventions_geojson)

print(interventions_geojson)

{"type": "FeatureCollection", "features": [{"id": "239", "type": "Feature", "properties": {"State B": "CHN", "State B code": "710", "styear": 1945, "endyear": 1949.0, "Name": "Operation Beleaguer", "Description": "China. In October 50,000 U.S. Marines were sent to North China to assist Chinese Nationalist authorities in disarming and repatriating the Japanese in China and in controlling ports, railroads, and airfields. This was in addition to approximately 60,000 U.S. forces remaining in China at the end of World War II.", "Era": 3.0, "PresName": "Truman", "Objective": "Build Foreign Regime Authority; Social Protection", "ForeignReg": 0.0, "BuildReg": 1.0, "Empire": 0.0, "Territory": 0.0, "Policy": 0.0, "SocialProt": 1.0, "Economic": 0.0, "Fatalities": null, "UNRegN": 142.0, "UNRegT": "Asia", "PresElectionYear": 0.0, "iso2c": "CN", "countryName": "China", "lat": 35.000082819944836, "long": 105.00008281994484}, "geometry": {"type": "Point", "coordinates": [105.00008281994484, 35.0000828

In [103]:
# Convert to new csv file
df.to_csv("interventions.csv")

## Using OpenAI to generate granular, event-level coordinates

In [9]:
df = pd.read_csv("interventions.csv")

In [132]:
from openai import OpenAI
from pydantic import BaseModel, Field, field_validator
import instructor
from dotenv import load_dotenv
import os

load_dotenv()

client = instructor.from_openai(OpenAI(api_key=os.getenv("API_KEY")))

In [111]:
class Coordinates(BaseModel):
    lat: float
    long: float

In [126]:
def get_coordinates(row):
    response = client.create(
        model="gpt-4o-mini",
        messages = [
            {
                "role": "system",
                "content": "You are an expert in identifying locations of historical military interventions based on descriptions and date ranges."
            },
            {
                "role": "user",
                "content": f"Provide the decimal degrees latitude and longitude for the following US military intervention event: {row['Name']}. "
                           f"Relevant Country: {row['countryName']}, Description: {row['Description']}, Start Year: {row['styear']}, End Year: {row['endyear']}."
            }
        ],
        response_model = Coordinates,
        temperature=0.2
    )
    
    return response.lat, response.long

In [116]:
example_data = {
    "Name": ['Korean War'], 
    "countryName": ['North Korea']
    "styear": [1950], 
    "endyear": [1953],
    "Description": "Korean War--NYT",}

get_coordinates(example_data)

(37.5, 126.5)

In [123]:
df[['lat', 'long']] = df.apply(get_coordinates, axis=1, result_type='expand')

In [124]:
import geopandas as gpd
from shapely.geometry import Point

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.long, df.lat))

ai_interventions_geojson = gdf.to_json()

with open('ai_interventions.geojson', 'w') as f:
    f.write(ai_interventions_geojson)

In [125]:
df.to_csv("ai_interventions.csv")

## Visualizations

In [88]:
# Are military interventions increasing over time?
df['decade'] = (df['styear'] // 10) * 10  # Create a new column for decades

# Group by decade, 'Era', and 'PresName'
year_df = df.groupby(['decade', 'Era', 'PresName']).size().reset_index(name='interv_count')

chart = alt.Chart(year_df).mark_bar().encode(
    x=alt.X('decade:O',
            axis=alt.Axis(
                title='Decade',
                format='d',
                labelAngle=0,
            )),
    y=alt.Y('interv_count:Q', axis=alt.Axis(title='Count')),
).properties(
    title='US heightens interventionism starting in the 1980s',
    width=270  # Set the width of the chart to 270 pixels
).configure_axis(
    grid=False  # Disable gridlines
)


chart

alt.Chart(...)

In [92]:
# Which US President was the most interventionist
pres_df = df.groupby('PresName').size().reset_index(name='interv_count')

pres_order = [
    'Truman', 'Eisenhower', 'Kennedy',
    'LBJ', 'Nixon', 'Ford',
    'Carter', 'Reagan', 'GHW',
    'Clinton', 'GWB', 'Obama', 'Trump'
]

chart = alt.Chart(pres_df).mark_bar().encode(
    x=alt.X('PresName:N',
            axis=alt.Axis(
                title='US President',
                labelAngle=45,
            ), sort = pres_order),
    y=alt.Y('interv_count:Q', axis=alt.Axis(title='Count')),
).properties(
    title='Clinton, Bush Jr., and Reagan the most interventionist',
    width=270  # Set the width of the chart to 270 pixels
)

chart

alt.Chart(...)

In [64]:
# Trend: which area of the world was the most intervened in?
reg_df = df.groupby('UNRegT').size().reset_index(name='interv_count')

alt.Chart(reg_df).mark_arc(innerRadius=50).encode(
    theta="interv_count:Q",
    color="UNRegT:N",
).properties(
    title='Global South disproportionately affected by US interventions',
    width=270  # Set the width of the chart to 270 pixels
)

alt.Chart(...)

In [66]:
# Beeswarm chart: what was the objective of these interventions?
objective_df = df[['ForeignReg', 'BuildReg', 'Empire', 'Territory', 'Policy', 'SocialProt', 'Economic']].sum().reset_index(name='interv_count')

alt.Chart(objective_df).mark_arc(innerRadius=50).encode(
    theta="interv_count:Q",
    color="index:N",
).properties(
    title='Building Foreign Regime Authority is the most common objective',
    width=270  # Set the width of the chart to 270 pixels
)

alt.Chart(...)

In [94]:
era_objectives_df = df.groupby('decade')[['ForeignReg', 'BuildReg', 'Empire', 'Territory', 'Policy', 'SocialProt', 'Economic']].sum().reset_index()
melted_df = era_objectives_df.melt(id_vars='decade', var_name='Objective', value_name='Interv_Count')

stacked_bar_chart = alt.Chart(melted_df).mark_bar().encode(
    x=alt.X('decade:N', title='Decade'),
    y=alt.Y('Interv_Count:Q', title='Intervention Count', stack='zero'),
    tooltip=['decade:N', 'Objective:N', 'Interv_Count:Q']
).properties(
    title='Interventions by Decade',
    width=270
).configure_axis(
    grid=False  # Disable gridlines
)


stacked_bar_chart

alt.Chart(...)

In [87]:
continent_objectives_df = df.groupby('UNRegT')[['ForeignReg', 'BuildReg', 'Empire', 'Territory', 'Policy', 'SocialProt', 'Economic']].sum().reset_index()
continent_objectives_df
melted_df = continent_objectives_df.melt(id_vars='UNRegT', var_name='Objective', value_name='Interv_Count')

pie_charts = []
for region in melted_df['UNRegT'].unique():
    region_df = melted_df[melted_df['UNRegT'] == region]
    pie_chart = alt.Chart(region_df).mark_arc().encode(
        theta=alt.Theta('Interv_Count:Q', stack=True),
        color='Objective:N',
        tooltip=['Objective:N', 'Interv_Count:Q']
    ).properties(
        title=region,
        width=200,
        height=200
    )
    pie_charts.append(pie_chart)

# Combine all pie charts into a single chart
combined_pie_charts = alt.vconcat(*pie_charts)

combined_pie_charts

alt.VConcatChart(...)

In [75]:
fatalities_df = df.groupby(['UNRegT', 'Fatalities']).size().reset_index(name='interv_count')

aggregated_df = fatalities_df.groupby('UNRegT').agg(
    total_fatalities=('Fatalities', 'sum'),
    total_interventions=('interv_count', 'sum')
).reset_index()

# Rename columns if desired for clarity
aggregated_df.rename(columns={
    'UNRegT': 'Continent',
    'total_fatalities': 'Total Fatalities',
    'total_interventions': 'Total Interventions'
}, inplace=True)

# Display the new DataFrame
print(aggregated_df)

melted_df = aggregated_df.melt(id_vars='Continent', value_vars=['Total Fatalities', 'Total Interventions'],
                                var_name='Type', value_name='Count')

stacked_bar_chart = alt.Chart(melted_df).mark_bar().encode(
    x=alt.X('Continent:N', title='Continent'),
    y=alt.Y('Count:Q', title='Count'),
    color='Type:N',
    tooltip=['Continent:N', 'Type:N', 'Count:Q']
).properties(
    title='Total Fatalities and Interventions by Continent',
    width=400,
    height=300
)

stacked_bar_chart

                   Continent  Total Fatalities  Total Interventions
0                     Africa          337034.0                   43
1                       Asia         6346166.0                   90
2                     Europe          274673.0                   34
3  Latin America & Caribbean          111618.0                   40
4                 N. America               0.0                    1


alt.Chart(...)